In [17]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

nltk.download('punkt')

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflowjs as tfjs
import pandas as pd
import pickle
import random
import json

[nltk_data] Downloading package punkt to /home/simon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [23]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print(np.shape(train_x[0]))

(82,)


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

2022-01-15 15:37:07.292083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-15 15:37:07.292242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-15 15:37:07.292290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-15 15:37:07.292319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-15 15:37:07.292348: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [11]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [12]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 0s 918us/step - loss: 2.2800 - accuracy: 0.0889
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1719 - accuracy: 0.0889
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1888 - accuracy: 0.1778
Epoch 4/200
9/9 [==============================] - 0s 841us/step - loss: 2.0870 - accuracy: 0.2667
Epoch 5/200
1/9 [==>...........................] - ETA: 0s - loss: 2.0259 - accuracy: 0.2000

2022-01-15 15:37:44.333897: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


9/9 [==============================] - 0s 707us/step - loss: 1.9492 - accuracy: 0.4222
Epoch 6/200
9/9 [==============================] - 0s 939us/step - loss: 1.8079 - accuracy: 0.5333
Epoch 7/200
9/9 [==============================] - 0s 753us/step - loss: 1.8357 - accuracy: 0.4889
Epoch 8/200
9/9 [==============================] - 0s 932us/step - loss: 1.7086 - accuracy: 0.5111
Epoch 9/200
9/9 [==============================] - 0s 703us/step - loss: 1.6964 - accuracy: 0.6000
Epoch 10/200
9/9 [==============================] - 0s 880us/step - loss: 1.3777 - accuracy: 0.6667
Epoch 11/200
9/9 [==============================] - 0s 842us/step - loss: 1.4052 - accuracy: 0.6222
Epoch 12/200
9/9 [==============================] - 0s 849us/step - loss: 1.2072 - accuracy: 0.7556
Epoch 13/200
9/9 [==============================] - 0s 664us/step - loss: 1.0320 - accuracy: 0.8222
Epoch 14/200
9/9 [==============================] - 0s 727us/step - loss: 0.9012 - accuracy: 0.8222
Epoch 15/200
9/9 

In [24]:
# save model to file
model.save("ai_simon.h5")

# export for JS
tfjs.converters.save_keras_model(model, "model_for_js")

with open("model_for_js/classes.json", "w") as classes_file:
    classes_file.write(json.dumps(classes))

with open("model_for_js/words.json", "w") as words_file:
    words_file.write(json.dumps(words))

In [14]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "katana-assistant-data.pkl", "wb" ) )